# Generate Resnet50 Models


In [1]:
import os
import time
import shutil
import tensorflow as tf
import tensorflow.neuron as tfn
import tensorflow.compat.v1.keras as keras
from tensorflow.keras.applications.resnet50 import ResNet50

Prepare export directories for compile/non-compiled versions of the model.

In [2]:
model_dir = "resnet50"
compiled_model_dir = model_dir + "_neuron"
shutil.rmtree(model_dir, ignore_errors=True)
shutil.rmtree(compiled_model_dir, ignore_errors=True)

Instantiate a Keras ResNet50 model.

In [3]:
keras.backend.set_learning_phase(0)
keras.backend.set_image_data_format('channels_last')
model = ResNet50(weights='imagenet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Export the model as SavedModel.

In [4]:
tf.saved_model.simple_save(
    session            = keras.backend.get_session(),
    export_dir         = model_dir,
    inputs             = {'input': model.inputs[0]},
    outputs            = {'output': model.outputs[0]})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: resnet50/saved_model.pb


And then compile it for Inferentia to be used on only one Neuron core. `--static-weights` option is used to cache all weights onto the neuron core's memory.

In [7]:
compiler_args = ['--static-weights', '--num-neuroncores', '1']
batch_size = 1
tfn.saved_model.compile(model_dir, compiled_model_dir, batch_size)

INFO:tensorflow:Restoring parameters from resnet50/variables/variables
INFO:tensorflow:Froze 320 variables.
INFO:tensorflow:Converted 320 variables to const ops.
INFO:tensorflow:fusing subgraph neuron_op_d6f098c01c780733 with neuron-cc
INFO:tensorflow:Number of operations in TensorFlow session: 4638
INFO:tensorflow:Number of operations after tf.neuron optimizations: 556
INFO:tensorflow:Number of operations placed on Neuron runtime: 554
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: resnet50_neuron/saved_model.pb
INFO:tensorflow:Successfully converted resnet50 to resnet50_neuron


{'OnNeuronRatio': 0.9964028776978417}